In [18]:
from collections import defaultdict
from contextlib import contextmanager


class CtxManager:
    def __init__(self, name):
        self.stats = defaultdict(int)

    def __enter__(self):
        print("in enter")
        return self.stats

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"exit with {self.stats=}")
        print("errors:", exc_type, exc_val, exc_tb)
        

@contextmanager
def calc_stat():
    stats = defaultdict(int)
    print("before yield")
    try:
        yield stats
    except Exception as err:
        print("ERRROR:", err)
        raise
    finally:
        print(f"exit with {stats=}")

In [11]:
with CtxManager("qwerty") as stats:
    print(f"ctx: {stats=}")
    stats["q"] += 1
    stats["t"] = 250
    #stats[{1, 2, 3}] = 42

print("end")

in enter
ctx: stats=defaultdict(<class 'int'>, {})
exit with self.stats=defaultdict(<class 'int'>, {'q': 1, 't': 250})
errors: None None None
end


In [16]:
with calc_stat() as stats:
    print(f"ctx: {stats=}")
    stats["q"] += 1
    stats["t"] = 250
    #stats[{1, 2, 3}] = 42

print("end")

before yield
ctx: stats=defaultdict(<class 'int'>, {})
exit with stats=defaultdict(<class 'int'>, {'q': 1, 't': 250})
end


In [17]:
with calc_stat() as stats:
    print(f"ctx: {stats=}")
    stats["q"] += 1
    stats["t"] = 250
    stats[{1, 2, 3}] = 42

print("end")

before yield
ctx: stats=defaultdict(<class 'int'>, {})
ERRROR: unhashable type: 'set'
exit with stats=defaultdict(<class 'int'>, {'q': 1, 't': 250})
end


In [21]:
with calc_stat() as stats:
    print(f"ctx: {stats=}")
    stats["q"] += 1
    stats["t"] = 250
    stats[{1, 2, 3}] = 42

print("end")

before yield
ctx: stats=defaultdict(<class 'int'>, {})
ERRROR: unhashable type: 'set'
exit with stats=defaultdict(<class 'int'>, {'q': 1, 't': 250})


TypeError: unhashable type: 'set'

In [22]:
class StatError(Exception):
    pass

class ZeroStatError(StatError):
    pass

In [39]:
try:
    raise ZeroStatError()
except ZeroStatError as err:
    print("zero", err)
    raise StatError from err
except StatError as err:
    print("stat", err)
except Exception as err:
    print("exp", err)
else:
    print("else")
finally:
    print("finally")


zero 
finally


StatError: 

In [40]:
try:
    raise Exception()
except ZeroStatError as err:
    print("zero", err)
    raise StatError from err
except StatError as err:
    print("stat", err)
except Exception as err:
    print("exp", err)
else:
    print("else")
finally:
    print("finally")


exp 
finally


In [42]:
!ls

README.md      bfm_loader.py  lesson-01      lesson-04
__pycache__    class_02.ipynb lesson-02      lesson-05
bfm.jsonl      class_05.ipynb lesson-03      test.py


In [98]:
students = ["s1\n", "s2\n", "s3\n"]

with open("students.txt", "w") as f:
    f.writelines(students)
    
with open("students.txt", "a+") as f:
    f.writelines(["s4\n", "s5\n"])

with open("students.txt", "a+") as f:
    f.write("\n".join(["s6", "s7"]))

In [63]:
!cat students.txt

s1
s2
s3
s4
s5
s6
s7

In [51]:
"\n".join(["s6", "s7"])

's6\ns7'

In [71]:
with open("students.txt") as f:
    print(f)
    print(f.fileno())

    for i, line in enumerate(f, 1):
        print(i, repr(line), line.strip())

<_io.TextIOWrapper name='students.txt' mode='r' encoding='UTF-8'>
72
1 's1\n' s1
2 's2\n' s2
3 's3\n' s3
4 's4\n' s4
5 's5\n' s5
6 's6\n' s6
7 's7' s7


In [74]:
with open("students.txt") as f:
    data = f.read()
    print(data)

s1
s2
s3
s4
s5
s6
s7


In [76]:
with open("students.txt") as f:
    data = f.read(10)
    print(repr(data))
    print(data)

's1\ns2\ns3\ns'
s1
s2
s3
s


In [78]:
with open("students.txt", "rb") as f:
    for i, line in enumerate(f, 1):
        print(i, repr(line), line.strip())

1 b's1\n' b's1'
2 b's2\n' b's2'
3 b's3\n' b's3'
4 b's4\n' b's4'
5 b's5\n' b's5'
6 b's6\n' b's6'
7 b's7' b's7'


In [79]:
with open("students.txt", "rb") as f:
    f.seek(10)
    print(f.read())

b'4\ns5\ns6\ns7'


In [102]:
with open("students.txt", "rb+") as f:
    f.seek(10)
    f.write(b"99")

In [103]:
!cat students.txt

s1
s2
s3
s99s5
s6
s7

In [101]:
def gen_file_chunks(name, batch_size):
    with open(name) as f:
        i = 0
        batch = []

        for line in f:
            batch.append(line)

            i += 1
            if i == batch_size:
                yield batch
                batch = []
                i = 0
        if batch:
            yield batch
  

for chunk in gen_file_chunks("students.txt", 2):
    print(chunk)

['s1\n', 's2\n']
['s3\n', 's4\n']
['s5\n', 's6\n']
['s7']
